In [1]:
import rospy
import json
import rospkg
from os.path import join
from itertools import permutations
from numpy import mean
from numpy import std
import numpy as np
import pylab as plt
import matplotlib.patches as mpatches
from itertools import combinations
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from scipy.stats.mstats import kruskalwallis
from scipy.stats import sem

In [2]:
res_folder = 'results_pre_study'

In [3]:
res_folder = 'results'

In [4]:
def plot_results(data, savefile):
    width = 0.25
    ind = np.arange(12) + 1

    # extract data
    mean = {}
    ste = {}
    for c in cond_list:
        mean[c] = []
        ste[c] = []
        for i in ind:
            mean[c].append(data[c]['Q' + str(i)]['mean'])
            ste[c].append(data[c]['Q' + str(i)]['sem'])

    fig = plt.figure(facecolor="white")
    ax = fig.add_subplot(111)
    plt.rcParams['font.size'] = 20

    rects1 = ax.bar(ind - 3 * width / 2, mean['fixed'], width, color='tomato', label='fixed')
    (line, caps, _) = plt.errorbar(ind - width, mean['fixed'], ste['fixed'], capsize=3, elinewidth=2,  ecolor='k')

    line.remove()
    for cap in caps:
        cap.set_color('k')
        cap.set_markeredgewidth(3)

    rects2 = ax.bar(ind - width / 2, mean['relative'], width, color='darkseagreen', label='relative')
    (line, caps, _) = plt.errorbar(ind, mean['relative'], ste['relative'], capsize=3, elinewidth=2,  ecolor='k')
    
    line.remove()
    for cap in caps:
        cap.set_color('k')
        cap.set_markeredgewidth(3)

    rects3 = ax.bar(ind + width / 2, mean['reba'], width, color='cornflowerblue', label='optimized')
    (line, caps, _) = plt.errorbar(ind + width, mean['reba'], ste['reba'], capsize=3, elinewidth=2,  ecolor='k')

    line.remove()
    for cap in caps:
        cap.set_color('k')
        cap.set_markeredgewidth(3)

    plt.plot((0, 13), (0, 0), 'k--',lw=2)
    
    legend = ax.legend(loc='lower left')
    # Set the fontsize
    for label in legend.get_texts():
        label.set_fontsize('small')

    for label in legend.get_lines():
        label.set_linewidth(1.5)  # the legend line width

    # add some text for labels, title and axes ticks
    ax.set_ylabel('Average Score')
    ax.set_ylim([-6, 6])
    y_tick = np.arange(13) - 6
    y_label = [str(x) for x in y_tick]
    plt.yticks(y_tick, y_label, rotation='vertical')
    
    labels = []
    for i in ind:
        labels.append('Q' + str(i))
    plt.xticks(ind, labels, rotation='vertical')
    ax.set_xlim([0, 13])

    # save svg file
    fig.set_size_inches(12.8, 10.24)
    plt.savefig(savefile, dpi=100, facecolor=fig.get_facecolor(), transparent=False)
    plt.close()

In [31]:
def test_differrences(data):
    t_test_res = {}
    min_q = min([len(data[c]['questions'][0]) for c in cond_list])
    min_gr = min([len(data[c]['gr0']) for c in cond_list])
    min_sum = min([len(data[c]['sum']) for c in cond_list])
    #min_q = -1
    #min_gr = -1
    #min_sum = -1
    for comb in comb_list:
        t_test_res[comb[0] + '_' + comb[1]] = {}
        t_test_res[comb[0] + '_' + comb[1]]['p_values'] = []
        for i in range(len(data[comb[0]]['questions'])):
            # remove size when number of sample is equal
            #t, p = ttest_ind(data[comb[0]]['questions'][i][:min_q], data[comb[1]]['questions'][i][:min_q])
            t, p = mannwhitneyu(data[comb[0]]['questions'][i][:min_q], data[comb[1]]['questions'][i][:min_q])
            #t, p = kruskalwallis(data[comb[0]]['questions'][i][:min_q], data[comb[1]]['questions'][i][:min_q])
            t_test_res[comb[0] + '_' + comb[1]]['p_values'].append(p)
        for i in range(3):
            #t, p = ttest_ind(data[comb[0]]['gr' + str(i)][:min_gr], data[comb[1]]['gr' + str(i)][:min_gr])
            t, p = mannwhitneyu(data[comb[0]]['gr' + str(i)][:min_gr], data[comb[1]]['gr' + str(i)][:min_gr])
            #t, p = kruskalwallis(data[comb[0]]['gr' + str(i)][:min_gr], data[comb[1]]['gr' + str(i)][:min_gr])
            t_test_res[comb[0] + '_' + comb[1]]['gr' + str(i)] = p
        #t, p = ttest_ind(data[comb[0]]['sum'][:min_sum], data[comb[1]]['sum'][:min_sum])
        t, p = mannwhitneyu(data[comb[0]]['sum'][:min_sum], data[comb[1]]['sum'][:min_sum])
        #t, p = kruskalwallis(data[comb[0]]['sum'][:min_sum], data[comb[1]]['sum'][:min_sum])
        t_test_res[comb[0] + '_' + comb[1]]['sum'] = p
    return t_test_res

In [6]:
rospy.init_node('survey_analysis')

In [7]:
rospack = rospkg.RosPack()
res_dir = join(rospack.get_path('thr_xp_human_comfort'), res_folder)
filename = join(res_dir, 'survey.json')
with open(filename) as datafile:
    survey_data = json.load(datafile)

In [8]:
cond_list = ['reba', 'relative', 'fixed']
comb_list = list(combinations(cond_list, 2))
perm_list = list(permutations(cond_list, 2))

In [9]:
results = {}
results['order'] = {}
results['order']['fixed'] = 0
results['order']['reba'] = 0
results['order']['relative'] = 0

results['questions'] = {}
results['questions']['A'] = {}
results['questions']['B'] = {}
for c in cond_list:
    results['questions'][c] = {}
    results['questions']['A'][c] = {}
    results['questions']['B'][c] = {}
    for i in range(1, 13):
        results['questions'][c]['Q' + str(i)] = {}
        results['questions'][c]['Q' + str(i)]['data'] = []
        results['questions']['A'][c]['Q' + str(i)] = {}
        results['questions']['A'][c]['Q' + str(i)]['data'] = []
        results['questions']['B'][c]['Q' + str(i)] = {}
        results['questions']['B'][c]['Q' + str(i)]['data'] = []
        
for p in perm_list:
    results['questions'][p[0] + '_' + p[1]] = [[],[]]

# add dict to check number of people in each conditions
results['conditions'] = {}
for c in cond_list:
    results['conditions'][c] = 0
for p in perm_list:
    results['conditions'][p[0] + '-' + p[1]] = 0.

In [10]:
# get total number of user
results['nb_users'] = len(survey_data)
# go through all results
for user_data in survey_data:
    user_id = str(user_data['ID'])
    # open config file of the user
    filename = join(res_dir, user_id, 'user_config.json')
    with open(filename) as datafile:
        user_config = json.load(datafile)
    user_cond = user_config['conditions']

    if user_config['laterality'] == 'right':
        # add conditions
        for c in user_cond:
            results['conditions'][c] += 1
        results['conditions'][user_cond[0] + '-' + user_cond[1]] += 1

        # get preference order
        if user_data['O1[1]'] == 'A':
            results['order'][user_cond[0]] += 1
        else:
            results['order'][user_cond[1]] += 1

        code_list = ['A', 'B']
        # get survey responses
        for i in range(2):
            for j in range(1, 13):
                q_score = (-1)**(j-1) * user_data['Q' + code_list[i] + '1[SQ' + code_list[i] + str(j) + ']']
                results['questions'][user_cond[i]]['Q' + str(j)]['data'].append(q_score)
                results['questions'][code_list[i]][user_cond[i]]['Q' + str(j)]['data'].append(q_score)
                results['questions'][user_cond[0] + '_' + user_cond[1]][i].append(q_score)

In [11]:
nb_user_cond = min([results['conditions']['reba'],
               results['conditions']['relative'],
               results['conditions']['fixed']])

In [12]:
print nb_user_cond, results['nb_users']

22 37


In [13]:
# calculate global satisfaction and check statistical differences
data_test = {}
data_test['A'] = {}
data_test['B'] = {}
for c in cond_list:
    data_test[c] = {}
    data_test[c]['questions'] = []
    data_test[c]['sum'] = []
    data_test['A'][c] = {}
    data_test['A'][c]['questions'] = []
    data_test['A'][c]['sum'] = []
    data_test['B'][c] = {}
    data_test['B'][c]['questions'] = []
    data_test['B'][c]['sum'] = []
    for i in range(3):
        data_test[c]['gr' + str(i)] = []
        data_test['A'][c]['gr' + str(i)] = []
        data_test['B'][c]['gr' + str(i)] = []
    results['questions'][c]['satisfaction'] = 0.
    for i in range(1, 13):
        q = results['questions'][c]['Q' + str(i)]
        qA = results['questions']['A'][c]['Q' + str(i)]
        qB = results['questions']['B'][c]['Q' + str(i)]
        data_test[c]['questions'].append(q['data'])
        data_test[c]['sum'] += q['data']
        data_test['A'][c]['questions'].append(qA['data'])
        data_test['A'][c]['sum'] += qA['data']
        data_test['B'][c]['questions'].append(qB['data'])
        data_test['B'][c]['sum'] += qB['data']
        if i < 5:
            data_test[c]['gr0'] += q['data']
            data_test['A'][c]['gr0'] += qA['data']
            data_test['B'][c]['gr0'] += qB['data']
        elif i < 9:
            data_test[c]['gr1'] += q['data']
            data_test['A'][c]['gr1'] += qA['data']
            data_test['B'][c]['gr1'] += qB['data']
        else:
            data_test[c]['gr2'] += q['data']
            data_test['A'][c]['gr2'] += qA['data']
            data_test['B'][c]['gr2'] += qB['data']
        # calculate mean of results
        q['mean'] = mean(q['data'])
        q['sem'] = sem(q['data'])
        qA['mean'] = mean(qA['data'])
        qA['sem'] = sem(qA['data'])
        qB['mean'] = mean(qB['data'])
        qB['sem'] = sem(qB['data'])
        results['questions'][c]['satisfaction'] += q['mean']
    results['questions'][c]['satisfaction'] /= 12

In [14]:
plot_results(results['questions'], join(res_dir, 'questions_results.svg'))
plot_results(results['questions']['A'], join(res_dir, 'questions_results_A.svg'))
plot_results(results['questions']['B'], join(res_dir, 'questions_results_B.svg'))

In [32]:
t_test_res = test_differrences(data_test)
t_test_A = test_differrences(data_test['A'])
t_test_B = test_differrences(data_test['B'])
for c in cond_list:
    t, p = mannwhitneyu(data_test['A'][c]['sum'], data_test['A'][c]['sum'])
    print c + ': ' + str(p)
    for i in range(3):
        t, p = mannwhitneyu(data_test['A'][c]['gr' + str(i)], data_test['A'][c]['gr' + str(i)])
        print c + ' gr' + str(i) + ': ' + str(p)
    print '----------------------------------'

reba: 0.49967350794
reba gr0: 0.498300237579
reba gr1: 0.498301609201
reba gr2: 0.498308602399
----------------------------------
relative: 0.49967566045
relative gr0: 0.4983191434
relative gr1: 0.498307925438
relative gr2: 0.498312280666
----------------------------------
fixed: 0.499745798364
fixed gr0: 0.498674540149
fixed gr1: 0.498686605623
fixed gr2: 0.498676309389
----------------------------------


In [33]:
# test differences between permutations
t, p = mannwhitneyu(results['questions']['reba_fixed'][0], results['questions']['reba_relative'][0])
print 'optimized_fixed & optimized_relative: ' + str(p)
t, p = mannwhitneyu(results['questions']['fixed_reba'][1], results['questions']['relative_reba'][1])
print 'fixed_optimized & relative_optimized: ' + str(p)
print '----------------------'
t, p = mannwhitneyu(results['questions']['relative_fixed'][0], results['questions']['relative_reba'][0])
print 'relative_fixed & relative_optimized: ' + str(p)
t, p = mannwhitneyu(results['questions']['fixed_relative'][1], results['questions']['reba_relative'][1])
print 'fixed_relative & optimized_relative: ' + str(p)
print '----------------------'
t, p = mannwhitneyu(results['questions']['fixed_reba'][0], results['questions']['fixed_relative'][0])
print 'fixed_optimized & fixed_relative: ' + str(p)
t, p = mannwhitneyu(results['questions']['reba_fixed'][1], results['questions']['relative_fixed'][1])
print 'relative_fixed & optimized_fixed: ' + str(p)

print '-------------------'
print '-------------------'

reba_fixed =  results['questions']['reba_fixed'][0] + results['questions']['fixed_reba'][1]
reba_relative = results['questions']['reba_relative'][0] + results['questions']['relative_reba'][1]
relative_fixed =  results['questions']['relative_fixed'][0] + results['questions']['fixed_relative'][1]
relative_reba = results['questions']['relative_reba'][0] + results['questions']['reba_relative'][1]
fixed_relative =  results['questions']['fixed_relative'][0] + results['questions']['relative_fixed'][1]
fixed_reba = results['questions']['fixed_reba'][0] + results['questions']['reba_fixed'][1]

t, p = mannwhitneyu(reba_fixed, reba_relative)
print 'optimized: ' + str(p)
t, p = mannwhitneyu(relative_fixed, relative_reba)
print 'relative: ' + str(p)
t, p = mannwhitneyu(fixed_reba, fixed_relative)
print 'fixed: ' + str(p)

print '-------------------'
print '-------------------'

t, p = mannwhitneyu(reba_relative, reba_fixed)
print 'reba in both: ' + str(p)
t, p = mannwhitneyu(fixed_reba, relative_reba)
print 'fixed & relative: ' + str(p)

optimized_fixed & optimized_relative: 0.496297916098
fixed_optimized & relative_optimized: 0.0364791330809
----------------------
relative_fixed & relative_optimized: 0.0101486076612
fixed_relative & optimized_relative: 5.86094949602e-05
----------------------
fixed_optimized & fixed_relative: 0.449021858671
relative_fixed & optimized_fixed: 0.0550563970093
-------------------
-------------------
optimized: 0.0884206058264
relative: 6.20094448631e-06
fixed: 0.153026068635
-------------------
-------------------
reba in both: 0.0884206058264
fixed & relative: 0.000504056076558


In [34]:
test_data = t_test_res

In [35]:
for key, value in test_data.iteritems():
    p_values = value['p_values']
    value['not_independant'] = {}
    value['not_independant']['question'] = []
    value['not_independant']['p_value'] = []
    for i in range(len(p_values)):
        if p_values[i] > 0.05:
            value['not_independant']['question'].append(i + 1)
            value['not_independant']['p_value'].append(p_values[i])

In [36]:
for key, value in test_data.iteritems():
    print key + ': ' + str(value['not_independant']['question'])
    print value['not_independant']['p_value']

reba_fixed: [6]
[0.27656539967967397]
reba_relative: [1, 3, 4, 6, 7, 10]
[0.05041298899397588, 0.068076138309199619, 0.21278194112756904, 0.27895899303784555, 0.12678317203512812, 0.065670740181230744]
relative_fixed: [6]
[0.14073173222717894]


In [37]:
for c in cond_list:
    print c + ': ' + str(results['questions'][c]['satisfaction'])

reba: 1.15909090909
relative: 0.604166666667
fixed: -0.892361111111


In [38]:
for key,value in test_data.iteritems():
    print key + ': ' + str(value['sum'])
    for i in range(3):
        print key + ' gr' + str(i) + ': ' + str(value['gr' + str(i)])
    print '----------------------------------'

reba_fixed: 1.75654987313e-11
reba_fixed gr0: 1.07314608432e-05
reba_fixed gr1: 0.00123993921105
reba_fixed gr2: 9.39154495314e-06
----------------------------------
reba_relative: 0.0645777401402
reba_relative gr0: 0.221137904662
reba_relative gr1: 0.337951633095
reba_relative gr2: 0.0812428069624
----------------------------------
relative_fixed: 4.30928253952e-10
relative_fixed gr0: 6.96658261428e-05
relative_fixed gr1: 0.000747854487889
relative_fixed gr2: 0.000141496013747
----------------------------------
